In [1]:
%pip install twilio


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/1.9 MB 4.8 MB/s eta 0:00:01
   --------------------------------- ------ 1.6/1.9 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 4.0 MB/s eta 0:00:00


In [ ]:
from typing import Dict, List
from datetime import datetime, timedelta
import json
from fastapi import FastAPI, Request
from twilio.rest import Client
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.memory import ConversationBufferMemory
from langchain.prompts import StringPromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, Field

class MedicalChatbot:
    def __init__(self):
        self.app = FastAPI()
        self.setup_routes()
        self.load_configurations()
        self.setup_agent()
        
    def load_configurations(self):
        """Carrega configurações necessárias para o funcionamento do chatbot"""
        self.config = {
            "twilio_account_sid": "seu_sid",
            "twilio_auth_token": "seu_token",
            "openai_api_key": "sua_chave",
            "clinic_info": {
                "nome": "Clínica Exemplo",
                "horario_funcionamento": "08:00 às 18:00",
                "especialidades": ["Clínica Geral", "Pediatria", "Cardiologia"]
            }
        }
        self.twilio_client = Client(
            self.config["twilio_account_sid"], 
            self.config["twilio_auth_token"]
        )

    def setup_agent(self):
        """Configura o agente inteligente com suas ferramentas e capacidades"""
        tools = [
            Tool(
                name="Agendamento",
                func=self.schedule_appointment,
                description="Agenda consultas médicas verificando disponibilidade"
            ),
            Tool(
                name="Informações",
                func=self.get_clinic_info,
                description="Fornece informações sobre a clínica e especialidades"
            ),
            Tool(
                name="Cancelamento",
                func=self.cancel_appointment,
                description="Cancela consultas agendadas"
            )
        ]
        
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        
        llm = ChatOpenAI(
            temperature=0.7,
            model_name="gpt-4",
            openai_api_key=self.config["openai_api_key"]
        )
        
        self.agent_executor = AgentExecutor.from_agent_and_tools(
            agent=self.create_agent(llm, tools),
            tools=tools,
            memory=self.memory,
            verbose=True
        )

    def create_agent(self, llm, tools):
        """Cria um agente personalizado com prompt específico para área médica"""
        template = """Você é um assistente virtual de uma clínica médica.
        Seja sempre cordial e profissional. Use linguagem clara e objetiva.
        
        Histórico do chat: {chat_history}
        Mensagem do paciente: {input}
        
        Pense passo a passo:
        1) Identifique a intenção principal do paciente
        2) Escolha a ferramenta mais adequada
        3) Execute a ação necessária
        
        Ferramentas disponíveis:
        {tools}
        
        Formato da resposta:
        Ação: a ação que você escolheu
        Pensamento: seu raciocínio
        """
        
        prompt = StringPromptTemplate(
            template=template,
            tools_getter=lambda x: x,
            input_variables=["input", "chat_history", "tools"]
        )
        
        return LLMSingleActionAgent(
            llm_chain=LLMChain(llm=llm, prompt=prompt),
            allowed_tools=[tool.name for tool in tools]
        )

    async def schedule_appointment(self, details: str) -> str:
        """Agenda consultas verificando disponibilidade"""
        # Implementação do agendamento
        # Verificaria disponibilidade no banco de dados
        return "Consulta agendada com sucesso para a data solicitada."

    def get_clinic_info(self, query: str) -> str:
        """Retorna informações sobre a clínica"""
        return json.dumps(self.config["clinic_info"], ensure_ascii=False)

    async def cancel_appointment(self, appointment_id: str) -> str:
        """Cancela agendamentos existentes"""
        # Implementação do cancelamento
        return "Consulta cancelada com sucesso."

    def setup_routes(self):
        """Configura as rotas da API para webhook do WhatsApp"""
        @self.app.post("/webhook")
        async def webhook(request: Request):
            form_data = await request.form()
            message = form_data.get("Body", "")
            sender = form_data.get("From", "")
            
            # Processa a mensagem através do agente
            response = await self.process_message(message, sender)
            
            # Envia resposta via WhatsApp
            self.twilio_client.messages.create(
                body=response,
                from_="whatsapp:+seu_numero",
                to=sender
            )
            return {"status": "success"}

    async def process_message(self, message: str, sender: str) -> str:
        """Processa mensagens recebidas usando o agente inteligente"""
        try:
            response = await self.agent_executor.arun(
                input=message,
                sender=sender
            )
            return response
        except Exception as e:
            return "Desculpe, ocorreu um erro ao processar sua mensagem. Por favor, tente novamente."

if __name__ == "__main__":
    import uvicorn
    chatbot = MedicalChatbot()
    uvicorn.run(chatbot.app, host="0.0.0.0", port=8000)